In [1]:
import pandas as pd
import numpy as np
import re
import random
from sklearn.utils import shuffle
import tensorflow as tf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from afinn import Afinn
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Input
import tensorflow_hub as hub

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import BertTokenizer, TFBertModel
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras import layers, models, optimizers
from itertools import product
from imblearn.over_sampling import SMOTE
from transformers import BertTokenizer
from itertools import product
import csv

from keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Activation

from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import auc

from tensorflow.keras import layers, models, optimizers
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

from keras_tuner import GridSearch as kt

#Import data
parlvote = pd.read_csv('ParlVote_concat.csv', header=None)
first_row = parlvote.iloc[0].tolist()
parlvote.columns = first_row
parlvote = parlvote.drop(index=0)

#########################BEGIN SAMPLE##################################
#parlvote =  parlvote.sample(n=300, random_state=42)
##########################END SAMPLE###################################

### Train for domain specific lexion ####
# Preprocess
# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the set of stopwords
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    # Tokenize and lowercase
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords and lemmatize
    return [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token.isalpha()]

# Apply preprocessing to the 'speech' column and store the result in 'speech_processed'
parlvote['speech_processed'] = parlvote['speech'].apply(preprocess_text)

parlvote['speech_svm'] = parlvote['speech_processed'].apply(' '.join)

# Train Word2Vec model
model = Word2Vec(parlvote['speech_processed'], vector_size=300, window=6, min_count=10, workers=4)
model.train(parlvote['speech_processed'], total_examples=len(parlvote['speech_processed']), epochs=5)

######## REMOVE speeches with >512 tokens ##########
rem_token = []
sh_pv_before = parlvote.shape

## Instantiate a BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")


# Tokenize the speeches and count tokens
parlvote['token_length'] = parlvote['speech'].apply(lambda x: len(tokenizer.tokenize(x)))

# Filter out rows with more than 512 tokens
max_tokens = 512
parlvote = parlvote[parlvote['token_length'] <= max_tokens]

# Now 'filtered_parlvote' is a DataFrame containing only rows with no more than 512 BERT tokens in the 'speech' column
# Shape after removing neutral
sh_pv_after = parlvote.shape
rem_token.append({"original" : sh_pv_before, ">512 removed" : sh_pv_after})
rem_token_df = pd.DataFrame(rem_token)

# Save the new DataFrame to a CSV file
rem_token_df.to_csv("removed_512.csv", index=False)


##### GENERATE SENTIMENT LABELS ########
### Domain Specific ####


def sentiment_score(speech, model, positive_seeds, negative_seeds):
    # Tokenize the speech
    words = speech.split()
    
    score = 0
    for word in words:
        if word in model.wv:
            word_vector = model.wv[word]
            
            # Sum of cosine similarities with positive seed words
            for seed in positive_seeds:
                if seed in model.wv:
                    seed_vector = model.wv[seed]
                    score += np.dot(word_vector, seed_vector) / (np.linalg.norm(word_vector) * np.linalg.norm(seed_vector))
                    
            # Subtract sum of cosine similarities with negative seed words
            for seed in negative_seeds:
                if seed in model.wv:
                    seed_vector = model.wv[seed]
                    score -= np.dot(word_vector, seed_vector) / (np.linalg.norm(word_vector) * np.linalg.norm(seed_vector))
    
    return score

positive_seeds = ["good", "excellent", "correct", "best", "happy", "positive", "fortunate"]
negative_seeds = ["bad", "terrible", "wrong", "worst", "disappointed", "negative", "unfortunate"]

# Generate raw sentiment scores
raw_sentiment_scores = [sentiment_score(speech, model, positive_seeds, negative_seeds) for speech in parlvote['speech']]

# Reshape the scores to fit the scaler input
raw_sentiment_scores = np.array(raw_sentiment_scores).reshape(-1, 1)

# Create a MinMaxScaler object with feature range of -1 to 1
scaler = MinMaxScaler(feature_range=(-1, 1))

# Fit the scaler to the data and transform the data
scaled_sentiment_scores = scaler.fit_transform(raw_sentiment_scores)

# Create 'domain_sentiment' column in the parlvote dataframe
parlvote['domain_sentiment'] = scaled_sentiment_scores.flatten()


###### BREAK #########
# if 1 == 1:
#     raise ValueError("Stopping execution at this cell")


### Standard Lexicons ###

# Initialize sentiment analyzers
vader_analyzer = SentimentIntensityAnalyzer()
afinn_analyzer = Afinn()
textblob_analyzer = TextBlob

# Define a function to generate the sentiment scores for each row
def generate_sentiment_scores(row):
    text = row['speech']
    
    # Generate VADER sentiment score
    vader_scores = vader_analyzer.polarity_scores(text)['compound']
    
    # Generate Afinn sentiment score
    afinn_score = afinn_analyzer.score(text)
    
    # Generate TextBlob sentiment score
    textblob_score = textblob_analyzer(text).sentiment.polarity
    
    return pd.Series([vader_scores, textblob_score, afinn_score])


# Apply the function to generate sentiment scores for each row and store the scores in new columns
parlvote[['vader', 'textblob', 'afinn']] = parlvote.apply(generate_sentiment_scores, axis=1)


# Define a function to convert VADER sentiment scores to labels
def vader_score_to_label(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Define a function to convert TextBlob sentiment scores to labels
def textblob_score_to_label(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

# Define a function to convert Afinn sentiment scores to labels
def afinn_score_to_label(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'


# Apply the label conversion functions to the sentiment score columns
parlvote['vader'] = parlvote['vader'].apply(vader_score_to_label)
parlvote['textblob'] = parlvote['textblob'].apply(textblob_score_to_label)
parlvote['afinn'] = parlvote['afinn'].apply(afinn_score_to_label)
parlvote['domain_sentiment'] = parlvote['domain_sentiment'].apply(textblob_score_to_label)

######### REMOVE 'NEUTRAL' LABELS #########
# Shape before removing neutral
removed = []
before = parlvote.shape

# remove instances where a row contains 'neutral'
pv = parlvote.loc[~((parlvote['vader'] == 'Neutral') | (parlvote['textblob'] == 'Neutral') | (parlvote['afinn'] == 'Neutral') | (parlvote['domain_sentiment'] == 'Neutral'))]
# Create a copy of the DataFrame to avoid modifying the original DataFrame
pv = pv.copy()

# Shape after removing neutral
after = pv.shape
removed.append({"original" : before, "neutral removed" : after})
removed_df = pd.DataFrame(removed)

# Save the new DataFrame to a CSV file
removed_df.to_csv("removed_neutral.csv", index=False)


######## CHANGE TO BINARY #########
def label_to_binary(label):
    if label == 'Positive':
        return 1
    else:
        return 0


def str_to_int(label):
    if label == '1':
        return 1
    else:
        return 0


# Convert sentiment labels to binary values for 'vader', 'textblob', and 'afinn' columns
pv['vader'] = pv['vader'].apply(label_to_binary)
pv['textblob'] = pv['textblob'].apply(label_to_binary)
pv['afinn'] = pv['afinn'].apply(label_to_binary)
pv['domain_sentiment'] = pv['domain_sentiment'].apply(label_to_binary)

# Convert the string of 'vote' to integer
pv['vote'] = pv['vote'].apply(str_to_int)


####### EMBEDDINGS ########
# Load a pre-trained BERT model
embedding_model = SentenceTransformer('bert-base-cased')
# Generate embeddings for the 'speech' column
pv['embeddings'] = parlvote['speech'].apply(lambda x: embedding_model.encode(x, convert_to_numpy=True))
print("Shape of the first embedding: ", pv['embeddings'].iloc[0].shape)
# Now 'parlvote' has a new column 'embeddings' containing the embeddings as numpy arrays


# # Define the label columns
# label_columns = ["vote", "vader", "textblob", "afinn", "domain_sentiment"]

# # Initialize an empty dictionary to store the distributions
# distributions = {}

# # For each label column, compute the distribution and store it in the dictionary
# for label in label_columns:
#     distributions[label] = pv[label].value_counts(normalize=True) * 100

# # Convert the distributions dictionary into a DataFrame
# distributions_df = pd.DataFrame(distributions)

# # Transpose the DataFrame so that each row corresponds to a label
# distributions_df = distributions_df.transpose()

# # Save the distributions DataFrame to a CSV file
# distributions_df.to_csv("distributions.csv")


###### ONE-HOT ENCODE PV (train-val) AND PV_TEST #######
# Define a function to one-hot encode a column
def one_hot_encode(df, column):
    df[column + '_positive'] = (df[column] == 1).astype(int)
    df[column + '_negative'] = (df[column] == 0).astype(int)
    return df


# Apply the function to one-hot encode the columns in the 'pv' dataframe
pv = one_hot_encode(pv, 'vote')
pv = one_hot_encode(pv, 'vader')
pv = one_hot_encode(pv, 'textblob')
pv = one_hot_encode(pv, 'afinn')
pv = one_hot_encode(pv, 'domain_sentiment')

# # Apply the function to one-hot encode the columns in the 'pv_test' dataframe
# pv_test = one_hot_encode(pv_test, 'vote')
# pv_test = one_hot_encode(pv_test, 'vader')
# pv_test = one_hot_encode(pv_test, 'textblob')
# pv_test = one_hot_encode(pv_test, 'afinn')
# pv_test = one_hot_encode(pv_test, 'domain_sentiment')

# # Prepare sliding window and sentence embedding functions
# def sliding_window(text, window_size, stride):
#     num_chunks = max((len(text) - window_size) // stride + 1, 1)
#     chunks = [text[i*stride:i*stride + window_size] for i in range(num_chunks)]
#     return chunks


# def get_sentence_embeddings(embedding_model, data, max_length, window_size, stride, aggregation='mean'):
#     all_embeddings = []
    
#     for text in data:
#         if len(text) > max_length:
#             chunks = sliding_window(text, window_size, stride)
#         else:
#             chunks = [text]

#         # Generate embeddings for each chunk
#         chunk_embeddings = embedding_model.encode(chunks)
        
#         # Aggregate the embeddings
#         if aggregation == 'mean':
#             aggregated_embedding = np.mean(chunk_embeddings, axis=0)
#         elif aggregation == 'max':
#             aggregated_embedding = np.max(chunk_embeddings, axis=0)
#         elif aggregation == 'min':
#             aggregated_embedding = np.min(chunk_embeddings, axis=0)
#         else:
#             raise ValueError(f"Unsupported aggregation method: {aggregation}")

#         all_embeddings.append(aggregated_embedding)

#     return np.array(all_embeddings)

# # Set the parameters for the sliding window
# max_length = 512
# window_size = 400
# stride = 200

def print_majority_baselines(df, one_hot_encoded_columns):
    for columns in one_hot_encoded_columns:
        majority_class = df[columns].sum(axis=0).idxmax()
        majority_baseline = df[majority_class].mean()
        
        print(f"Majority baseline for {', '.join(columns)}:")
        print(f"  Majority class: {majority_class}")
        print(f"  Baseline accuracy: {majority_baseline:.2%}\n")

# usage
one_hot_encoded_columns = [
    ['vote_positive', 'vote_negative'],
    ['vader_positive', 'vader_negative'],
    ['textblob_positive', 'textblob_negative'],
    ['afinn_positive', 'afinn_negative'], 
    ['domain_sentiment_positive', 'domain_sentiment_negative']
]
##### PRINT MAJORITY BASELINES FOR PV ######
print_majority_baselines(pv, one_hot_encoded_columns)

#### PRINT MAJORITY BASELINES FOR PV_TEST #####
def majority_baseline(df, columns):
        majority_class = df[columns].sum(axis=0).idxmax()
        majority_baseline = df[majority_class].mean()
        return majority_baseline

vote_majority = majority_baseline(pv, one_hot_encoded_columns[0])
vader_majority = majority_baseline(pv, one_hot_encoded_columns[1])
textblob_majority = majority_baseline(pv, one_hot_encoded_columns[2])
afinn_majority = majority_baseline(pv, one_hot_encoded_columns[3])
domain_majority = majority_baseline(pv, one_hot_encoded_columns[4])

# Convert one-hot encoded columns to a NumPy array
vote_labels = pv[['vote_positive', 'vote_negative']].values
vader_labels = pv[['vader_positive', 'vader_negative']].values
textblob_labels = pv[['textblob_positive', 'textblob_negative']].values
afinn_labels = pv[['afinn_positive', 'afinn_negative']].values
domain_labels = pv[['domain_sentiment_positive', 'domain_sentiment_negative']].values

C:\Users\beren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:44: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beren\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\beren\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyboardInterrupt: 

In [ ]:
def sentiment_analysis_pipeline(data, train_labels, test_labels, lstm_units, spatial_dropout_rate, name, dense_units, learning_rate, batch_size, epochs, input_dropout_rate, recurrent_dropout_rate, output_dropout_rate, smote, use_kfold, test_size=0.2, n_splits=5):
    data = np.stack(data, axis=0)  # Convert the lists of embeddings to a NumPy array
    
    if use_kfold:
        # K-fold cross-validation
        kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        splits = kfold.split(data, np.argmax(train_labels, axis=1))
    else:
        train_data, test_data, train_labels, _ = train_test_split(data, train_labels, test_size=test_size, stratify=vote_labels, random_state=42)
        test_labels = test_labels[len(train_labels):]
        splits = [(range(len(train_data)), range(len(test_data)))]

    # Store evaluation scores for each fold or train-test split
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for train_index, test_index in splits:
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = train_labels[train_index], test_labels[test_index]

        # Create the SMOTE instance if specified
        if smote:
            smote_sampler = SMOTE(random_state=42)
            X_train, y_train = smote_sampler.fit_resample(X_train, y_train)

        # Define the model
        model = models.Sequential([
            layers.Input(shape=(X_train.shape[1],)),
            layers.Reshape((1, X_train.shape[1])),
            layers.SpatialDropout1D(spatial_dropout_rate),
            layers.Bidirectional(layers.LSTM(lstm_units, dropout=input_dropout_rate, recurrent_dropout=recurrent_dropout_rate)),
            layers.Dense(dense_units, activation='relu'),
            layers.Dropout(output_dropout_rate),
            layers.Dense(y_train.shape[1], activation='softmax')
        ])


        model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
        
        # Train the model
        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

        # Predict on test data
        y_pred = np.argmax(model.predict(X_test), axis=-1)
        y_test_decoded = np.argmax(y_test, axis=-1)

        train_val_name = 'Train-Validation Loss: ' + name
        # Plot training and validation loss
        plt.figure(figsize=(12, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(train_val_name)
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

        # Compute the confusion matrix
        confusion_mtx = confusion_matrix(y_test_decoded, y_pred)

        cf_name = 'Confusion Matrix: ' + name 
         # Plot the confusion matrix
        plt.figure(figsize=(10,8))
        sns.heatmap(confusion_mtx, annot=True, fmt="d", cmap='Blues')
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.title(cf_name)
        plt.show()

        # Print the classification report
        print(classification_report(y_test_decoded, y_pred))

        pr_name = 'Precision-Recall curve: ' + name
        # Plot Precision-Recall curve
        precision, recall, _ = precision_recall_curve(y_test_decoded, y_pred)
        plt.figure(figsize=(7,7))
        plt.plot(recall, precision, label='PR curve (area = %0.2f)' % auc(recall, precision))
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(pr_name)
        plt.legend(loc="lower right")
        plt.show()

        # Evaluation
        accuracy = accuracy_score(y_test_decoded, y_pred)
        precision = precision_score(y_test_decoded, y_pred, average='weighted')
        recall = recall_score(y_test_decoded, y_pred, average='weighted')
        f1 = f1_score(y_test_decoded, y_pred, average='weighted')

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Print average scores
    print(f"Average accuracy: {np.mean(accuracies):.2%}")
    print(f"Average precision: {np.mean(precisions):.2%}")
    print(f"Average recall: {np.mean(recalls):.2%}")
    print(f"Average F1-score: {np.mean(f1_scores):.2%}")
    accuracy = np.mean(accuracies)
    precision = np.mean(precisions)
    recall = np.mean(recalls)
    f1 = np.mean(f1_scores)
    return accuracy, precision, recall, f1


def mlp_baseline_pipeline(data, train_labels, test_labels, smote, use_kfold, name, test_size=0.2, n_splits=5, dense_units=100, learning_rate=1e-3, batch_size=32, epochs=10):
    data = np.stack(data, axis=0)  # Convert the lists of embeddings to a NumPy array
    if use_kfold:
        kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        splits = kfold.split(data, np.argmax(train_labels, axis=1))
    else:
        train_data, test_data, train_labels, _ = train_test_split(data, train_labels, test_size=test_size, stratify=vote_labels, random_state=42)
        test_labels = test_labels[len(train_labels):]
        splits = [(range(len(train_data)), range(len(test_data)))]

    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for train_index, test_index in splits:
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = train_labels[train_index], test_labels[test_index]

        if smote:
            smote_sampler = SMOTE(random_state=42)
            X_train, y_train = smote_sampler.fit_resample(X_train, y_train)

        model = Sequential([
            Dense(dense_units, input_shape=(X_train.shape[1],)),  # Hidden layer of 100 nodes
            BatchNormalization(),  # Batch normalization
            Activation('relu'),  # ReLU activation function
            Dropout(0.5),  # Dropout regularization rate of 0.5
            Dense(y_train.shape[1], activation='sigmoid')  # Sigmoid activation in the output layer
        ])

        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
        model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1)

        y_pred = np.argmax(model.predict(X_test), axis=-1)
        y_test_decoded = np.argmax(y_test, axis=-1)

        train_val_name = 'Train-Validation Loss: ' + name
        #PLOTTING
        # Plot training and validation loss
        plt.figure(figsize=(12, 4))
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(train_val_name)
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

        # Compute the confusion matrix
        confusion_mtx = confusion_matrix(y_test_decoded, y_pred)
        
        cf_name = 'Confusion Matrix: ' + name 

         # Plot the confusion matrix
        plt.figure(figsize=(10,8))
        sns.heatmap(confusion_mtx, annot=True, fmt="d", cmap='Blues')
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.title(cf_name)
        plt.show()

        # Print the classification report
        print(classification_report(y_test_decoded, y_pred))

        pr_name = 'Precision-Recall curve: ' + name
        # Plot Precision-Recall curve
        precision, recall, _ = precision_recall_curve(y_test_decoded, y_pred)
        plt.figure(figsize=(7,7))
        plt.plot(recall, precision, label='PR curve (area = %0.2f)' % auc(recall, precision))
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(pr_name)
        plt.legend(loc="lower right")
        plt.show()


        #EVALUATION METRICS
        accuracy = accuracy_score(y_test_decoded, y_pred)
        precision = precision_score(y_test_decoded, y_pred, average='weighted')
        recall = recall_score(y_test_decoded, y_pred, average='weighted')
        f1 = f1_score(y_test_decoded, y_pred, average='weighted')

        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    print(f"Average accuracy: {np.mean(accuracies):.2%}")
    print(f"Average precision: {np.mean(precisions):.2%}")
    print(f"Average recall: {np.mean(recalls):.2%}")
    print(f"Average F1-score: {np.mean(f1_scores):.2%}")
    accuracy = np.mean(accuracies)
    precision = np.mean(precisions)
    recall = np.mean(recalls)
    f1 = np.mean(f1_scores)
    return accuracy, precision, recall, f1



# ##### LOGISTIC REGRESSION #######
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import average_precision_score

# def logistic_regression_pipeline(data, train_labels, test_labels, smote, use_kfold, name, test_size=0.2, n_splits=5):
#     data = np.stack(data, axis=0)  # Convert the lists of embeddings to a NumPy array
#     le = LabelEncoder()
#     train_labels_encoded = le.fit_transform(np.argmax(train_labels, axis=1))
#     test_labels_encoded = le.transform(np.argmax(test_labels, axis=1))
#     if use_kfold:
#         kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
#         splits = kfold.split(data, train_labels_encoded)
#     else:
#         train_data, test_data, train_labels, _ = train_test_split(data, train_labels_encoded, test_size=test_size, stratify=train_labels_encoded, random_state=42)
#         test_labels = test_labels_encoded[len(train_labels):]
#         splits = [(range(len(train_data)), range(len(test_data)))]

#     accuracies = []
#     precisions = []
#     recalls = []
#     f1_scores = []

#     for train_index, test_index in splits:
#         X_train, X_test = data[train_index], data[test_index]
#         y_train, y_test = train_labels[train_index], test_labels[test_index]

#         if smote:
#             smote_sampler = SMOTE(random_state=42)
#             X_train, y_train = smote_sampler.fit_resample(X_train, y_train)

#         model = LogisticRegression(random_state=42, max_iter=1000)
#         model.fit(X_train, y_train)

#         y_pred = model.predict(X_test)

#         # Compute the confusion matrix
#         confusion_mtx = confusion_matrix(y_test, y_pred)
        
#         cf_name = 'Confusion Matrix: ' + name 

#          # Plot the confusion matrix
#         plt.figure(figsize=(10,8))
#         sns.heatmap(confusion_mtx, annot=True, fmt="d", cmap='Blues')
#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title(cf_name)
#         plt.show()

#         # Print the classification report
#         print(classification_report(y_test, y_pred))

#         pr_name = 'Precision-Recall curve: ' + name
#         # Plot Precision-Recall curve
#         precision, recall, _ = precision_recall_curve(y_test, model.predict_proba(X_test)[:,1])
#         plt.figure(figsize=(7,7))
#         plt.plot(recall, precision, label='PR curve (area = %0.2f)' % auc(recall, precision))
#         plt.xlabel('Recall')
#         plt.ylabel('Precision')
#         plt.title(pr_name)
#         plt.legend(loc="lower right")
#         plt.show()

#         #EVALUATION METRICS
#         accuracy = accuracy_score(y_test, y_pred)
#         precision = precision_score(y_test, y_pred, average='weighted')
#         recall = recall_score(y_test, y_pred, average='weighted')
#         f1 = f1_score(y_test, y_pred, average='weighted')

#         accuracies.append(accuracy)
#         precisions.append(precision)
#         recalls.append(recall)
#         f1_scores.append(f1)

#     print(f"Average accuracy: {np.mean(accuracies):.2%}")
#     print(f"Average precision: {np.mean(precisions):.2%}")
#     print(f"Average recall: {np.mean(recalls):.2%}")
#     print(f"Average F1-score: {np.mean(f1_scores):.2%}")
#     accuracy = np.mean(accuracies)
#     precision = np.mean(precisions)
#     recall = np.mean(recalls)
#     f1 = np.mean(f1_scores)
#     return accuracy, precision, recall, f1


# ##### SVM FUNCTION #####
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import Pipeline
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
# from sklearn.preprocessing import LabelEncoder

# def svm_pipeline(data, train_labels, test_labels, name, test_size=0.2):
    
#     train_data, test_data, train_labels, _ = train_test_split(data, train_labels, test_size=test_size, stratify=vote_labels, random_state=42)
#     test_labels = test_labels[len(train_labels):]
#     splits = [(range(len(train_data)), range(len(test_data)))]
    
#     accuracies = []
#     precisions = []
#     recalls = []
#     f1_scores = []
    
#     # It's a binary classification, LabelEncoder is used instead of LabelBinarizer
#     train_labels = np.argmax(train_labels, axis=1)
#     test_labels = np.argmax(test_labels, axis=1)
#     le = LabelEncoder()
#     train_labels = le.fit_transform(train_labels)
#     test_labels = le.transform(test_labels)

#     #data = data.values
#     # Define the pipeline
#     pipeline = Pipeline([
#         ('tfidf', TfidfVectorizer(max_df=0.9, min_df=5)),
#         ('clf', SVC(kernel='linear', probability=True, class_weight='balanced'))
#     ])

#     # Train the SVM classifier
#     pipeline.fit(train_data, train_labels)

#     y_pred = pipeline.predict(test_data)

#     #EVALUATION METRICS
#     accuracy = accuracy_score(test_labels, y_pred)
#     precision = precision_score(test_labels, y_pred)
#     recall = recall_score(test_labels, y_pred)
#     f1 = f1_score(test_labels, y_pred)
    
#     # Print the classification report
#     print(classification_report(test_labels, y_pred))

#     print(f"Accuracy: {accuracy:.2%}")
#     print(f"Precision: {precision:.2%}")
#     print(f"Recall: {recall:.2%}")
#     print(f"F1-score: {f1:.2%}")

#     return accuracy, precision, recall, f1

###############################################################################################

# # Hyperparameter optimizations
# search_space = {
#     'lstm_units': [32, 64, 128],
#     'spatial_dropout_rate': [0.1, 0.2, 0.3],
#     'dense_units': [16, 32, 64],
#     'learning_rate': [1e-3, 1e-4],
#     'batch_size': [16, 32],
#     'epochs': [5, 10, 15], 
#     'smote' : [True, False], 
#     'use_kfold' : [True, False]
# }

# # Hyperparameter setup


# def generate_combinations(search_space):
#     keys, values = zip(*search_space.items())
#     return [dict(zip(keys, v)) for v in product(*values)]

# combinations = generate_combinations(search_space)

# # Hyperparamter optimization functie
# def run_pipeline_with_combination(combination, data, train_labels, test_labels):
#     return sentiment_analysis_pipeline(
#         data=data,
#         train_labels=train_labels,
#         test_labels=test_labels,
#         smote=combination['smote'],
#         use_kfold=combination['use_kfold'],
#         lstm_units=combination['lstm_units'],
#         spatial_dropout_rate=combination['spatial_dropout_rate'],
#         dense_units=combination['dense_units'],
#         learning_rate=combination['learning_rate'],
#         batch_size=combination['batch_size'],
#         epochs=combination['epochs']
#     )

# # Define the filename for the CSV file
# filename = 'experiment_results.csv'

# # Define the header for the CSV file
# header = ['Experiment', 'Parameters', 'F1-score']

# # Initialize an empty list to store the results
# results = []

# # Hyperparameter optimization Experiment 1 
# best_combination_exp_1 = None
# best_f1_exp_1 = -1

# for combination in combinations:
#     f1_exp_1 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=vote_labels, test_labels=vote_labels)
    
#     if f1_exp_1 > best_f1_exp_1:
#         best_f1_exp_1 = f1_exp_1
#         best_combination_exp_1 = combination

# print(f"Best combination experiment 1: {best_combination_exp_1}")
# print(f"Best F1-score experiment 1: {best_f1_exp_1}")

# # Add the results for Experiment 1 to the results list
# results.append(('Experiment 1', best_combination_exp_1, best_f1_exp_1))

# # Hyperparameter optimization Experiment 2 
# best_combination_exp_2 = None
# best_f1_exp_2 = -1

# for combination in combinations:
#     f1_exp_2 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=vader_labels, test_labels=vader_labels)
    
#     if f1_exp_2 > best_f1_exp_2:
#         best_f1_exp_2 = f1_exp_2
#         best_combination_exp_2 = combination

# print(f"Best combination experiment 2: {best_combination_exp_2}")
# print(f"Best F1-score experiment 2: {best_f1_exp_2}")

# # Add the results for Experiment 2 to the results list
# results.append(('Experiment 2', best_combination_exp_2, best_f1_exp_2))

# # Hyperparameter optimization Experiment 3 
# best_combination_exp_3 = None
# best_f1_exp_3 = -1

# for combination in combinations:
#     f1_exp_3 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=vader_labels, test_labels=vote_labels)
    
#     if f1_exp_3 > best_f1_exp_3:
#         best_f1_exp_3 = f1_exp_3
#         best_combination_exp_3 = combination

# print(f"Best combination experiment 3: {best_combination_exp_3}")
# print(f"Best F1-score experiment 3: {best_f1_exp_3}")

# # Add the results for Experiment 3 to the results list
# results.append(('Experiment 3', best_combination_exp_3, best_f1_exp_3))

# # Hyperparameter optimization Experiment 4 
# best_combination_exp_4 = None
# best_f1_exp_4 = -1

# for combination in combinations:
#     f1_exp_4 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=textblob_labels, test_labels=textblob_labels)
    
#     if f1_exp_4 > best_f1_exp_4:
#         best_f1_exp_4 = f1_exp_4
#         best_combination_exp_4 = combination

# print(f"Best combination experiment 4: {best_combination_exp_4}")
# print(f"Best F1-score experiment 4: {best_f1_exp_4}")

# # Add the results for Experiment 4 to the results list
# results.append(('Experiment 4', best_combination_exp_4, best_f1_exp_4))

# # Hyperparameter optimization Experiment 5
# best_combination_exp_5 = None
# best_f1_exp_5 = -1

# for combination in combinations:
#     f1_exp_5 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=textblob_labels, test_labels=vote_labels)
    
#     if f1_exp_5 > best_f1_exp_5:
#         best_f1_exp_5 = f1_exp_5
#         best_combination_exp_5 = combination

# print(f"Best combination experiment 5: {best_combination_exp_5}")
# print(f"Best F1-score experiment 5: {best_f1_exp_5}")

# # Add the results for Experiment 5 to the results list
# results.append(('Experiment 5', best_combination_exp_5, best_f1_exp_5))

# # Hyperparameter optimization Experiment 6 
# best_combination_exp_6 = None
# best_f1_exp_6 = -1

# for combination in combinations:
#     f1_exp_6 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=afinn_labels, test_labels=afinn_labels)
    
#     if f1_exp_6 > best_f1_exp_6:
#         best_f1_exp_6 = f1_exp_6
#         best_combination_exp_6 = combination

# print(f"Best combination experiment 6: {best_combination_exp_6}")
# print(f"Best F1-score experiment 6: {best_f1_exp_6}")

# # Add the results for Experiment 6 to the results list
# results.append(('Experiment 6', best_combination_exp_6, best_f1_exp_6))

# # Hyperparameter optimization Experiment 7 
# best_combination_exp_7 = None
# best_f1_exp_7 = -1

# for combination in combinations:
#     f1_exp_7 = run_pipeline_with_combination(combination, data=pv['embeddings'], train_labels=afinn_labels, test_labels=vote_labels)
    
#     if f1_exp_7 > best_f1_exp_7:
#         best_f1_exp_7 = f1_exp_7
#         best_combination_exp_7 = combination

# print(f"Best combination experiment 7: {best_combination_exp_7}")
# print(f"Best F1-score experiment 7: {best_f1_exp_7}")

# # Add the results for Experiment 7 to the results list
# results.append(('Experiment 7', best_combination_exp_7, best_f1_exp_7))

# # Write the results to the CSV file
# with open(filename, 'a', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(header)
#     writer.writerows(results)


###############################################################################################

# Experiments with best parameters
# Define the filename for the CSV file
filename_2 = 'best_experiment_results.csv'

# Define the header for the CSV file
header_2 = ['Experiment', 'Model', 'Train label', 'Test label', 'Accuracy', 'Precision', 'Recall', 'F1', 'Majority baseline', 'Parameters']

bilstm_parameters_human = "kfold=False, LSTM_units = 64, \n spatial_dropout = 0.3, \n dense_units = 64, \n 'learning_rate = 0.001, \n batch_size=100, \n epochs=10, \n  input_dropout_rate=0.3, \n recurrent_dropout_rate=0.3, \n output_dropout_rate=0.4"
bilstm_parameters_auto = "kfold=False, LSTM_units = 64, \n spatial_dropout = 0.2, \n dense_units = 64, \n 'learning_rate = 0.001, \n batch_size=100, \n epochs=10, \n  input_dropout_rate=0.2, \n recurrent_dropout_rate=0.2, \n output_dropout_rate=0.5"
bilstm_parameters_mix = "kfold=False, LSTM_units = 64, \n spatial_dropout = 0.3, \n dense_units = 64, \n 'learning_rate = 0.001, \n batch_size=100, \n epochs=10, \n  input_dropout_rate=0.3, \n recurrent_dropout_rate=0.3, \n output_dropout_rate=0.5"
mlp_parameters = 'Standard'


# Initialize an empty list to store the results
results_2 = []


## Experiment 1 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=vote_labels,
        test_labels=vote_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.0001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.4,
        name='Experiment 1 BiLSTM'
)


# Add the results for Experiment 1 - BiLSTM to the results list
results_2.append(('1', 'BiLSTM', 'Vote', 'Vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_human))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=vote_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.0001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False, 
    name='Experiment 1 MLP'
)


# Add the results for Experiment 1 - MLP baseline to the results list
results_2.append(('1', 'MLP', 'Vote', 'Vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Experiment 2
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=vader_labels,
        test_labels=vader_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.2,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.2, 
        recurrent_dropout_rate=0.2, 
        output_dropout_rate=0.5,
        name='Experiment 2 BiLSTM'
)


# Add the results for Experiment 2 - BiLSTM to the results list
results_2.append(('2', 'BiLSTM', 'VADER', 'VADER', accuracy, precision, recall, f1, vader_majority, bilstm_parameters_auto))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=vader_labels,
    test_labels=vader_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False,
    name='Experiment 2 MLP'
)


# Add the results for Experiment 2 - MLP baseline to the results list
results_2.append(('2', 'MLP', 'VADER', 'VADER', accuracy, precision, recall, f1, vader_majority, mlp_parameters))


## Experiment 3 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=vader_labels,
        test_labels=vote_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.5,
        name='Experiment 3 BiLSTM'
)

# Add the results for Experiment 3 - BiLSTM to the results list
results_2.append(('3', 'BiLSTM', 'VADER', 'Vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_mix))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=vader_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False,
    name='Experiment 3 MLP'
)

# Add the results for Experiment 3 - MLP baseline to the results list
results_2.append(('3', 'MLP', 'VADER', 'Vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Experiment 4 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=textblob_labels,
        test_labels=textblob_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.2,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.2, 
        recurrent_dropout_rate=0.2, 
        output_dropout_rate=0.5, 
        name='Experiment 4 BiLSTM'
)

# Add the results for Experiment 4 - BiLSTM to the results list
results_2.append(('4', 'BiLSTM', 'TextBlob', 'TextBlob', accuracy, precision, recall, f1, textblob_majority, bilstm_parameters_auto))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=textblob_labels,
    test_labels=textblob_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False, 
    name='Experiment 4 MLP'
)

# Add the results for Experiment 4 - MLP baseline to the results list
results_2.append(('4', 'MLP', 'TextBlob', 'TextBlob', accuracy, precision, recall, f1, textblob_majority, mlp_parameters))


## Experiment 5 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=textblob_labels,
        test_labels=vote_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.5, 
        name='Experiment 5 BiLSTM'
)

# Add the results for Experiment 5 - BiLSTM to the results list
results_2.append(('5', 'BiLSTM', 'TextBlob', 'Vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_mix))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=textblob_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False, 
    name='Experiment 5 MLP'
)

# Add the results for Experiment 5 - MLP baseline to the results list
results_2.append(('5', 'MLP', 'TextBlob', 'Vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Experiment 6 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=afinn_labels,
        test_labels=afinn_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.2,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.2, 
        recurrent_dropout_rate=0.2, 
        output_dropout_rate=0.5, 
        name='Experiment 6 BiLSTM'
)

# Add the results for Experiment 6 - BiLSTM to the results list
results_2.append(('6', 'BiLSTM', 'Afinn', 'Afinn', accuracy, precision, recall, f1, afinn_majority, bilstm_parameters_auto))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=afinn_labels,
    test_labels=afinn_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False, 
    name='Experiment 6 MLP'
)

# Add the results for Experiment 6 - MLP baseline to the results list
results_2.append(('6', 'MLP', 'Afinn', 'Afinn', accuracy, precision, recall, f1, afinn_majority, mlp_parameters))

## Experiment 7 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=afinn_labels,
        test_labels=vote_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.5, 
        name='Experiment 7 BiLSTM'
)

# Add the results for Experiment 7 - BiLSTM to the results list
results_2.append(('7', 'BiLSTM', 'Afinn', 'Vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_mix))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=afinn_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False, 
    name='Experiment 7 MLP'
)

# Add the results for Experiment 7 - MLP baseline to the results list
results_2.append(('7', 'MLP', 'Afinn', 'Vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))

## Experiment 8 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=domain_labels,
        test_labels=domain_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.2,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.2, 
        recurrent_dropout_rate=0.2, 
        output_dropout_rate=0.5, 
        name='Experiment 8 BiLSTM'
)

# Add the results for Experiment 8 - BiLSTM to the results list
results_2.append(('8', 'BiLSTM', 'Domain', 'Domain', accuracy, precision, recall, f1, domain_majority, bilstm_parameters_auto))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=domain_labels,
    test_labels=domain_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False, 
    name='Experiment 8 MLP'
)

# Add the results for Experiment 8 - MLP baseline to the results list
results_2.append(('8', 'MLP', 'Domain', 'Domain', accuracy, precision, recall, f1, domain_majority, mlp_parameters))


## Experiment 9 
accuracy, precision, recall, f1 = sentiment_analysis_pipeline(
        data=pv['embeddings'],
        train_labels=domain_labels,
        test_labels=vote_labels,
        smote=False,
        use_kfold=False,
        lstm_units=64,
        spatial_dropout_rate=0.3,
        dense_units=64,
        learning_rate=0.001,
        batch_size=100,
        epochs=10,
        input_dropout_rate=0.3, 
        recurrent_dropout_rate=0.3, 
        output_dropout_rate=0.5, 
        name='Experiment 9 BiLSTM'
)

# Add the results for Experiment 9 - BiLSTM to the results list
results_2.append(('9', 'BiLSTM', 'Domain', 'Vote', accuracy, precision, recall, f1, vote_majority, bilstm_parameters_mix))

accuracy, precision, recall, f1 = mlp_baseline_pipeline(
    data=pv['embeddings'],
    train_labels=domain_labels,
    test_labels=vote_labels,
    test_size=0.2,
    n_splits=5,  
    learning_rate=0.001,
    batch_size=32,
    epochs=10, 
    smote=False, 
    use_kfold=False, 
    name='Experiment 9 MLP'
)

# Add the results for Experiment 9 - MLP baseline to the results list
results_2.append(('9', 'MLP', 'Domain', 'Vote', accuracy, precision, recall, f1, vote_majority, mlp_parameters))


## Write the results to the CSV file
with open(filename_2, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header_2)
    writer.writerows(results_2)